In [37]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display
import polars as pl
from datetime import timedelta
import datetime 
import json 
import toml
import holidays
import sys

features = toml.load(r'C:\Users\N000193384\Documents\sncf_project\sncf_playground\data\features.toml')
times_cols = features['times_cols']
macro_horizon = features['MACRO_HORIZON']
p = Path(features['ABS_DATA_PATH'])
sys.path.insert(1, p)

from src.preprocessing.times import (
    from_day_to_time_fe,
    get_covid_table,
)
from src.preprocessing.quality import trim_timeseries, minimum_length_uid
from src.models.forecast.direct import DirectForecaster
from src.preprocessing.lags import get_significant_lags
from src.preprocessing.times import get_basic_holidays
from src.project_utils import load_data
from src.models.lgb_wrapper import GBTModel
from src.preprocessing.validation import freeze_validation_set
from src.preprocessing.lags import compute_autoreg_features

ts_uid = features["ts_uid"]
date_col = features['date_col']
y = features['y']
submit = False 
flist = features["flist"]
long_horizon = np.arange(macro_horizon)
chains = np.array_split(long_horizon, 6)
exog = ["job", "ferie", "vacances"] + times_cols

with open('data/params.json', 'rb') as stream:
    params_q = json.load(stream)

in_dt = datetime.date(2020, 6, 1)

covid_df = get_covid_table(2015, 2024)
df_dates = df_dates = get_basic_holidays()
holidays_fe = list(filter(lambda x: date_col not in x, df_dates.columns))
covid_fe = list(filter(lambda x: date_col not in x, covid_df.columns))
exog = exog + holidays_fe + covid_fe

In [38]:
"""
train_data, test_data, submission = load_data(p)

test_data = (
    test_data.pipe(from_day_to_time_fe, time=date_col, frequency="day")
    .join(df_dates, how="left", on=[date_col])
    .join(covid_df, how="left", on=[date_col])
)

train_data = (
    train_data.pipe(trim_timeseries, target="y", uid=ts_uid, time=date_col)
    .pipe(from_day_to_time_fe, time=date_col, frequency="day")
    .join(df_dates, how="left", on=[date_col])
    .join(
        covid_df.with_columns(
            pl.lit(np.where(np.any(covid_df != 0, axis=1), 0.01, 1)).alias(
                "covid_weight"
            )
        ),
        how="left",
        on=[date_col],
    )
    # sncf strike | 2019-12-01 to 2021-11-01
    .filter(
        (pl.col(date_col) >= in_dt)
        & (pl.col(date_col) != pl.datetime(2019, 12, 1))
        & (pl.col(date_col) != pl.datetime(2021, 11, 1))
    )
)

good_ts = minimum_length_uid(
    train_data, uid=ts_uid, time=date_col, min_length=round(364 * 1.2)
)
train_data = train_data.filter(pl.col(ts_uid).is_in(good_ts))
# test.
left_term = train_data.select([date_col, ts_uid, "y"] + exog).with_columns(
    pl.lit(1).alias("train")
)
right_term = test_data.select([date_col, ts_uid, "y"] + exog).with_columns(
    pl.lit(0).alias("train")
)
full_data = pl.concat((left_term, right_term), how="vertical_relaxed")
del left_term, right_term


# define params
significant_lags = get_significant_lags(train_data, date_col=date_col, target=y)
significant_lags = [x for x in significant_lags if x%7 == 0 and x<= macro_horizon][1:]

autoreg_dict = {
        ts_uid: {
            "groups": ts_uid,
            "horizon": lambda horizon: np.int32(horizon),
            "wins": np.array([28, 56]),
            "shifts": lambda horizon: np.int32([horizon]),
            "lags": lambda horizon: np.array(significant_lags) + horizon,
            "funcs": np.array(flist),
        },
        "ts_uid_dow": {
            "groups": [ts_uid, "day_of_week"],
            "horizon": lambda horizon: np.int32(np.ceil(horizon / 7) + 1),
            "wins": np.array([4, 8]),
            "shifts": lambda horizon: np.int32([np.ceil(horizon / 7) + 1]),
            "lags": lambda horizon: np.arange(1, 7) + np.ceil(horizon / 7) + 1,
            "funcs": np.array(flist),
        }
    }

for key in autoreg_dict.keys():
    autoreg_dict[key]["horizon"] = autoreg_dict[key]["horizon"](macro_horizon)
    autoreg_dict[key]["shifts"] = autoreg_dict[key]["shifts"](macro_horizon)
    autoreg_dict[key]["lags"] = autoreg_dict[key]["lags"](macro_horizon)

cat_cols = ['week', 'month', 'day_of_week', "job", "ferie", "vacances"]
target = "y""""

SyntaxError: unterminated string literal (detected at line 75) (470988224.py, line 75)

In [ ]:
'''import os
from typing import Dict, List

import lightgbm as lgb
import numpy as np
import optuna
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


def objective(
    trial: optuna.trial,
    train_data: pd.DataFrame,
    exog: List,
    seed: int = 12345,
):
    """_summary_

    Args:
        trial (optuna.trial): _description_
        train_x (pd.DataFrame): _description_
        test (pd.DataFrame): _description_
        features (List): _description_
        target (sself, optional): _description_. Defaults to "".
        seed (int, optional): _description_. Defaults to 12345.

    Returns:
        _type_: _description_
    """
    optuna_params = {
        "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt", "dart"]),
        "objective": trial.suggest_categorical(
            "objective", ["regression", "huber", "regression_l1", "quantile"]
        ),
        "metric": trial.suggest_categorical("metric", ["rmse"]),
        "alpha": trial.suggest_categorical(
            "alpha",
            [0.5, 0.52, 0.55, 0.57, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.7, 0.69, 0.7],
        ),
        "force_row_wise": trial.suggest_categorical("force_row_wise", [True, False]),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05, log=False),
        "max_depth": trial.suggest_int("max_depth", 4, 15),
        "sub_row": trial.suggest_categorical("sub_row", [0.6, 0.7, 0.8, 1.0]),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
        "min_child_weight": trial.suggest_float("min_child_weight", 1e-3, 4, log=True),
        "min_child_samples": trial.suggest_float(
            "min_child_samples", 20, 5000, log=False
        ),
        "num_iterations": trial.suggest_int(
            "n_estimators",
            200,
            3000,
        ),
        "num_leaves": trial.suggest_int("num_leaves", 25, 800),
        "max_bins": trial.suggest_int("max_bins", 24, 1000),
        "min_data_in_bin": trial.suggest_int("min_data_in_bin", 25, 1000),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 5, 1000),
        "feature_fraction_seed": trial.suggest_categorical(
            "feature_fraction_seed", [seed]
        ),
        "bagging_seed": trial.suggest_categorical("bagging_seed", [seed]),
        "seed": trial.suggest_categorical("seed", [seed]),
        "verbose": trial.suggest_categorical("verbose", [-1]),
    }

    horizon = 181
    lags = [x for x in significant_lags if x <= horizon]
    win_list =  [ x for x in significant_lags if x % 7 == 0 and x <= horizon][1:] 

    autoreg_dict = {
        ts_uid : {
            'groups' : ts_uid,
            'horizon': lambda horizon : np.int32(horizon),
            'wins' : np.array(win_list), 
            'shifts' : lambda horizon : np.int32([horizon, horizon+28, horizon+56]), 
            'lags' : lambda horizon : np.array(significant_lags) + horizon,
            'funcs' : np.array(flist)
        },
        "ts_uid_dow" : {
            'groups':[ts_uid, 'day_of_week'],
            'horizon' : lambda horizon : np.int32(np.ceil(horizon /7) + 1),
            'wins' : np.array([4, 8, 12, 16, 20]), 
            'shifts' : lambda horizon : np.int32([np.ceil(horizon /7) + 1, np.ceil(horizon /7) + 4]), 
            'lags' : lambda horizon : np.arange(1, 7) + np.ceil(horizon/7)+1,
            'funcs' : np.array(flist)
        }
    }


    effect_m = GBTModel(params=optuna_params, 
                        early_stopping_value=200, 
                        features= None,
                        custom_loss=optuna_params["objective"], 
                        categorical_features=[]
                        )


    dir_forecaster = DirectForecaster(
        model=effect_m,
        ts_uid=ts_uid,
        forecast_range=np.arange(horizon),
        target_str="y",
        date_str="date",
        exogs=exog,
        features_params=autoreg_dict
    )

    dir_forecaster.fit(train_data=train_data)
    return dir_forecaster.evaluate()["mae"].values[0]


def parameters_tuning(
    initial_params: Dict,
    tuning_objective,
    n_trials: int = 25,
    njobs: int = -1,
):
    """parameter for tuning over sudy

    Args:
        tuning_objective (_type_): _description_
        n_trials (int, optional): _description_. Defaults to 25.

    Returns:
        _type_: _description_

    example :

    func = lambda trial: objective(trial=trial,
                                    train_x=train_x,
                                    test=residualised_test,
                                    covariates=covariates,
                                    target=y,
                                    seed=12345
                                    )
    study_df, best_params = parameters_tuning(tuning_objective=func, n_trials=25, initial_params={})
        print(best_params)
        print(study_df)
        study_df.to_csv('bparamslgb_new.csv', sep="|", index=False)
    """
    study = optuna.create_study(direction="minimize")
    # study.enqueue_trial(initial_params)
    study.optimize(tuning_objective, n_trials=n_trials, n_jobs=njobs)
    print("Number of finished trials:", len(study.trials))
    print("Best trial:", study.best_trial.params)
    study_df = study.trials_dataframe()
    return study_df, study.best_params


func = lambda trial: objective(trial=trial,
                                train_data=train_data,
                                exog=exog,
                                seed=12345
                                )
study_df, best_params = parameters_tuning(tuning_objective=func, n_trials=30, initial_params={}, njobs=10)
print(best_params)
print(study_df)
study_df.to_csv('bparamslgb_new.csv', sep="|", index=False)'''

In [95]:
import pandas as pd
import numpy as np

from typing import List, Union, Optional, Dict
from sklearn.linear_model import RidgeCV
from pygam import GAM

from joblib import Parallel, delayed
import polars as pl
from datetime import datetime
from statsmodels.tsa.deterministic import DeterministicProcess


class LogLinear:
    def __init__(
        self,
        features: List[str],
        target: str,
        ts_uid: str,
        n_jobs: int = -1,
        frequency: str = "D",
        use_gam: bool = True,
        date_col:str="date",
        ndays:int = 364,
        horizon:int=181
    ) -> None:
        self.n_jobs = n_jobs
        self.models = {}
        self.ts_uid = ts_uid
        self.spline_transformer = None
        self.target = target
        self.features = features
        self.date_col = date_col
        self.frequency = frequency
        self.use_gam = use_gam
        self.ndays = ndays
        self.horizon = horizon

    def basic_features(self, end_date) -> pl.DataFrame:
        index = pd.date_range(end_date - timedelta(days=self.ndays), end_date, freq=self.frequency)
        dp = DeterministicProcess(
            index=index, 
            constant=True, 
            order=1, 
            seasonal=True, 
            fourier=0, 
            additional_terms=(), 
            drop=True
            )
        return dp

    def fit_single(self, y, max_dt):
        dp = self.basic_features(end_date=max_dt)
        features = dp.in_sample()
        size = y.shape[0]
        training_obs = size if self.ndays >= size else self.ndays
        features = features[-training_obs:]
        y = y[-training_obs:]
        model = GAM(distribution="normal") if self.use_gam else RidgeCV(cv=4)
        model.fit(features, np.log1p(y))
        return (model, dp)

    def fit(self, data:pl.DataFrame) -> pl.DataFrame:
        self.all_keys = data[self.ts_uid].unique().to_list()
        results = Parallel(n_jobs=self.n_jobs)(
            delayed(self.fit_single)(
                y=df[self.target], 
                max_dt=df[self.date_col].max())
            for key, df in data.group_by([self.ts_uid])
        )
        for key, (model, dp) in zip(data[self.ts_uid].unique(), results):
            self.models[key] = (model, dp)

    def predict_single(self, key:str)-> pl.DataFrame:
        model, dp = self.models[str(key)]
        features = dp.out_of_sample(self.horizon)
        forecast = np.expm1(model.predict(features))
        output = pl.DataFrame()
        output = output.with_columns(
            pl.lit(forecast).alias("y_hat"),
            pl.lit(key[0]).alias(self.ts_uid),
            # pl.lit(X["date"]),
        )
        return output

    def predict(self) -> pl.DataFrame:
        predictions = Parallel(n_jobs=self.n_jobs)(
            delayed(self.predict_single)(key) for key in self.all_keys
        )
        results = pl.concat(predictions)
        return results


In [96]:
ll = LogLinear(
    features=['job', 'ferie', 'vacances'],
    target=y,
    ts_uid=ts_uid,
    use_gam=True,
    ndays=364*2, 
    horizon=181
)

ll.fit(train_data)
out = ll.predict()

AttributeError: 'DeterministicProcess' object has no attribute 'out_sample'

In [103]:
ll.models.get('BKJ')[1].out_of_sample(181)

,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)"
2023-01-01,1.0,730.0,1.0,0.0,0.0,0.0,0.0,0.0
2023-01-02,1.0,731.0,0.0,1.0,0.0,0.0,0.0,0.0
2023-01-03,1.0,732.0,0.0,0.0,1.0,0.0,0.0,0.0
2023-01-04,1.0,733.0,0.0,0.0,0.0,1.0,0.0,0.0
2023-01-05,1.0,734.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
2023-06-26,1.0,906.0,0.0,1.0,0.0,0.0,0.0,0.0
2023-06-27,1.0,907.0,0.0,0.0,1.0,0.0,0.0,0.0
2023-06-28,1.0,908.0,0.0,0.0,0.0,1.0,0.0,0.0
2023-06-29,1.0,909.0,0.0,0.0,0.0,0.0,1.0,0.0


# evaluate

In [18]:
base_col = [ts_uid, date_col, y, "day_of_year"]

from src.preprocessing.lags import reference_shift_from_day
from src.analysis.metrics import display_metrics

train_data, test_data, submission = load_data(p)


def freeze_validation_set(
    df: pl.DataFrame,
    date: str,
    val_size: int,
    return_train: bool = True,
) -> pl.DataFrame:
    max_dt = df[date].max()
    cut = max_dt - timedelta(days=val_size)
    valid = df.filter(pl.col(date) > cut)  # .select([ts_uid, date, target])
    if return_train:
        train = df.filter(pl.col(date) <= cut)
        return train, valid
    else:
        return valid
    

baseline = (train_data
    .pipe(from_day_to_time_fe, time="date", frequency="day")
    .pipe(reference_shift_from_day,    
        target_col=y, 
        ts_uid=ts_uid,
        dayofyear_col="day_of_year"       )
    .pipe(freeze_validation_set, return_train=False, date=date_col, val_size=181)
    .with_columns(
        pl.coalesce(
            pl.col('reference_y'), 
            pl.col('reference_y').mean().over(ts_uid),
            pl.col('reference_y').mean(),
            0
            )
    )
    .select(["index", date_col, ts_uid, y, "reference_y"])
)

del train_data, test_data
import gc
gc.collect()
import os 

In [21]:
direct_mean_global_valid = (pl.read_csv('out/global_log_chain_lgb.csv', separator=",", infer_schema_length=1000)
                                                 .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)
direct_mean_global_valid.shape

(195307, 3)

In [25]:
baseline[date_col].max()

datetime.date(2022, 12, 31)

In [32]:
direct_mean_global_valid.filter((pl.col(ts_uid) == "OYD") & (pl.col(date_col) == datetime.datetime(2022, 12, 28))).sort(by="date")

station,date,global_log_chain_y_hat
str,date,f64
"""OYD""",2022-12-28,52.332357
"""OYD""",2022-12-28,58.888227
"""OYD""",2022-12-28,70.471682
"""OYD""",2022-12-28,97.832968


In [33]:
all_files = list(filter(lambda s : ".csv" in s, os.listdir('out/')))
fcol = []
for files in all_files:
    print(files)
    if files not in ['global_direct_mlp.csv', 'nixtla_validation.csv', 
                     'prophet.csv', 'neural_validation.csv', 'mlp_forecast.csv']:
        fcst_col = files.replace('lgb', 'y_hat').replace('.csv', '')
        print(baseline.shape)
        baseline = baseline.join(
            (pl.read_csv(f'out/{files}', 
                        separator=",", 
                        infer_schema_length=1000,
                        columns=[ts_uid, date_col, files.replace('lgb', 'y_hat').replace('.csv', '')]
            ).with_columns(pl.col(date_col).cast(pl.Date).alias(date_col))
            .unique(subset=[ts_uid, date_col])
            ), 
            how="left", on=[ts_uid, date_col]
        )
        fcol.append(fcst_col)

global_direct_mlp.csv
global_log_chain_lgb.csv
(78102, 5)
global_log_direct_lgb.csv
(78102, 6)
global_mean_chain_lgb.csv
(78102, 7)
global_mean_direct_lgb.csv
(78102, 8)
global_median_chain_lgb.csv
(78102, 9)
global_median_direct_lgb.csv
(78102, 10)
global_None_chain_lgb.csv
(78102, 11)
global_None_direct_lgb.csv
(78102, 12)
global_rolling_mean_chain_lgb.csv
(78102, 13)
global_rolling_mean_direct_lgb.csv
(78102, 14)
global_rolling_median_chain_lgb.csv
(78102, 15)
global_rolling_median_direct_lgb.csv
(78102, 16)
global_rolling_zscore_chain_lgb.csv
(78102, 17)
global_rolling_zscore_direct_lgb.csv
(78102, 18)
global_shiftn_chain_lgb.csv
(78102, 19)
global_shiftn_direct_lgb.csv
(78102, 20)
local_log_direct_lgb.csv
(78102, 21)


ColumnNotFoundError: unable to find column "local_log_direct_y_hat"; valid columns: ["date", "station", "job", "ferie", "vacances", "index", "is_train", "y", "y_copy", "not_outlier", "year", "week", "month", "day", "day_of_year", "day_of_week", "quarter", "week_of_month", "is_weekend", "New Year's Day", "Easter Monday", "Labor Day", "Victory Day", "Ascension Day", "Whit Monday", "National Day", "Assumption Day", "All Saints' Day", "Armistice Day", "Christmas Day", "First Lockdown", "Second Lockdown", "Third Lockdown (Partial)", "covid_weight", "ar_y_lag181_station", "ar_y_lag188_station", "ar_y_lag195_station", "ar_y_lag202_station", "ar_y_lag209_station", "ar_y_lag216_station", "ar_y_lag230_station", "ar_y_lag272_station", "ar_y_lag300_station", "ar_y_win7_shift181_rolling_mean_station", "ar_y_win7_shift181_rolling_median_station", "ar_y_win7_shift181_rolling_std_station", "ar_y_win7_shift181_rolling_skew_station", "ar_y_win7_shift209_rolling_mean_station", "ar_y_win7_shift209_rolling

In [5]:



statsmodel_valid = (pl.read_csv('out/nixtla_validation.csv', separator=",", infer_schema_length=1000)
                    .rename({"unique_id":ts_uid, "ds":date_col})
                    .select([ts_uid, date_col, 'HoltWinters', 'AutoETS', "AutoARIMA",
                              'AutoTheta', 'AutoTBATS', 'arithmetic_forecast_ensamble']
                              ).with_columns(pl.col(date_col).cast(pl.Date).alias(date_col))
)


direct_mean_global_valid = (pl.read_csv('out/global_mean_direct_lgb.csv', separator=",", infer_schema_length=1000, 
                                        columns=[ts_uid, date_col, "global_direct_mean_y_hat"])
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

direct_rolling_zscore_global_valid = (pl.read_csv('out/global_rolling_zscore_direct_lgb.csv', separator=",", infer_schema_length=1000, 
                                                  columns=[ts_uid, date_col, "global_direct_rolling_zscore_y_hat"])
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

direct_med_global_valid = (pl.read_csv('out/global_median_direct_lgb.csv', separator=",", infer_schema_length=1000, 
                                       columns=[ts_uid, date_col, "global_direct_median_y_hat"])
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

direct_None_global_valid = (pl.read_csv('out/global_None_direct_lgb.csv', separator=",", infer_schema_length=1000, 
                                        columns=[ts_uid, date_col, "global_direct_None_y_hat"])
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

direct_log_global_valid = (pl.read_csv('out/global_log_direct_lgb.csv', separator=",", infer_schema_length=1000, 
                                       columns=[ts_uid, date_col, "global_direct_log_y_hat"])
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

direct_rolmean_global_valid = (pl.read_csv('out/global_rolling_mean_direct_lgb.csv', separator=",", infer_schema_length=1000, 
                                           columns=[ts_uid, date_col, "global_direct_rolling_mean_y_hat"])
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

direct_rolmed_global_valid = (pl.read_csv('out/global_rolling_median_direct_lgb.csv', separator=",", infer_schema_length=1000, 
                                          columns=[ts_uid, date_col, "global_direct_rolling_median_y_hat"])
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

direct_shiftn_global_valid = (pl.read_csv('out/global_shiftn_direct_lgb.csv', separator=",", infer_schema_length=1000, 
                                          columns=[ts_uid, date_col, "global_direct_shiftn_y_hat"]
)
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

# =======
chain_mean_global_valid = (pl.read_csv('out/global_mean_chain_lgb.csv', separator=",", infer_schema_length=1000, 
                                       columns=[ts_uid, date_col, "global_mean_chain_y_hat"])
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

chain_rolling_zscore_global_valid = (pl.read_csv('out/global_rolling_zscore_chain_lgb.csv', separator=",", infer_schema_length=1000, 
                                                 columns=[ts_uid, date_col, "global_rolling_zscore_chain_y_hat"])
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

chain_med_global_valid = (pl.read_csv('out/global_median_chain_lgb.csv', separator=",", infer_schema_length=1000, 
                                      columns=[ts_uid, date_col, "global_median_chain_y_hat"])
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

chain_None_global_valid = (pl.read_csv('out/global_None_chain_lgb.csv', separator=",", infer_schema_length=1000, 
                                       columns=[ts_uid, date_col, "global_None_chain_y_hat"])
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

chain_log_global_valid = (pl.read_csv('out/global_log_chain_lgb.csv', separator=",", infer_schema_length=1000, 
                                      columns=[ts_uid, date_col, "global_log_chain_y_hat"])
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

chain_rolmean_global_valid = (pl.read_csv('out/global_rolling_mean_chain_lgb.csv', separator=",", infer_schema_length=1000, 
                                          columns=[ts_uid, date_col, "global_rolling_mean_chain_y_hat"])
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

chain_rolmed_global_valid = (pl.read_csv('out/global_rolling_median_chain_lgb.csv', separator=",", infer_schema_length=1000, 
                                         columns=[ts_uid, date_col, "global_rolling_median_chain_y_hat"])
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

chain_shiftn_global_valid = (pl.read_csv('out/global_shiftn_chain_lgb.csv', separator=",", infer_schema_length=1000, 
                                         columns=[ts_uid, date_col, "global_shiftn_chain_y_hat"])
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col),
                                   # pl.col("global_direct_y_hat").exp()
                                   )
)

#==============


nfcst  = ["TimesNet"	,"NHITS"	,"NBEATSx"	, "PatchTST"	,"TSMixerx"	, "iTransformer", "ensamble_forecast"]
neural_fcst_model = pl.read_csv("out/neural_validation.csv", separator=",", infer_schema_length=1000) 
for col in nfcst:
    neural_fcst_model = neural_fcst_model.with_columns(pl.lit(np.expm1(neural_fcst_model[col])).alias(col))
neural_fcst_model = neural_fcst_model.rename({"unique_id":ts_uid, "ds":date_col, "ensamble_forecast":"deep_ensemble"})
neural_fcst_model = neural_fcst_model.with_columns(pl.col(date_col).cast(pl.Date))
nfcst = nfcst[:-1] + ["deep_ensemble"]

direct_local_valid = (pl.read_csv('out/local_direct_lgb.csv', separator=",", infer_schema_length=1000)
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col))
)


prophet_m = (pl.read_csv('out/prophet.csv', infer_schema_length=1000)
                     .with_columns(pl.col("ds").str.to_datetime().cast(pl.Date).alias(date_col))
                     .rename({"unique_id":ts_uid, "yhat":"prophet_yhat"})
)

prophet_m = prophet_m.with_columns(pl.lit(np.expm1(prophet_m['prophet_yhat'])))


mlp_global_valid  = (pl.read_csv('out/global_direct_mlp.csv', separator=",", infer_schema_length=1000)
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col))
                     .rename({"global_direct_y_hat":"global_direct_mlp_y_hat"})
)
mlp_global_valid = mlp_global_valid.with_columns(pl.lit(np.expm1(mlp_global_valid['global_direct_mlp_y_hat'])))

chain_global_valid = (pl.read_csv('out/global_chain_lgb.csv', separator=",", infer_schema_length=1000)
                     # .rename({"y_hat":"global_chain_y_hat"})
                     .with_columns(pl.col(date_col).cast(pl.Date).alias(date_col))
)

baseline = (train_data
    .pipe(from_day_to_time_fe, time="date", frequency="day")
    .pipe(reference_shift_from_day,    
        target_col=y, 
        ts_uid=ts_uid,
        dayofyear_col="day_of_year"       )
    .pipe(freeze_validation_set, return_train=False, date=date_col, val_size=181)
    .with_columns(
        pl.coalesce(
            pl.col('reference_y'), 
            pl.col('reference_y').mean().over(ts_uid),
            pl.col('reference_y').mean(),
            0
            )
    )
    .join( statsmodel_valid, how="left",  on=[ts_uid, date_col]    )
    """
    .join(
            direct_log_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            direct_mean_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            direct_med_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            direct_rolmean_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            direct_rolmed_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            direct_rolling_zscore_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            direct_shiftn_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            direct_None_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    """
    #=
    .join(
            chain_mean_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            chain_med_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            chain_rolmean_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            chain_rolmed_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            chain_rolling_zscore_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            chain_shiftn_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            chain_None_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )
    .join(
            chain_log_global_valid,
            how="left", 
            on=[ts_uid, date_col]
        )

    .join(mlp_global_valid.select([ts_uid, date_col, 'global_direct_mlp_y_hat']), how="left", on = [ts_uid, date_col])
    .join(neural_fcst_model, how="left", on = [ts_uid, date_col])
    .join(prophet_m, how="left", on = [ts_uid, date_col])
    )

forecast_cols = [
                'HoltWinters', 'AutoETS', "AutoARIMA",
                 'AutoTheta', 'AutoTBATS', "prophet_yhat",
                 'arithmetic_forecast_ensamble',
                 "reference_y",

                 "global_direct_mlp_y_hat", 
                
                 
                 # "global_direct_mean_y_hat", "global_direct_shiftn_y_hat",
                 # "global_direct_median_y_hat", "global_direct_rolling_mean_y_hat", "global_direct_rolling_median_y_hat", 
                 # "global_direct_None_y_hat", "global_direct_log_y_hat", "global_direct_rolling_zscore_y_hat",
                 
                 
                 "global_chain_mlp_y_hat", "global_mean_chain_y_hat", "global_shiftn_chain_y_hat",
                 "global_median_chain_y_hat", "global_rolling_median_chain_y_hat", "global_rolling_mean_chain_y_hat", 
                 "global_None_chain_y_hat", "global_None_chain_y_hat", "global_rolling_zscore_chain_y_hat"
                 ] + nfcst



"""baseline = baseline.with_columns(
    pl.concat_list(["reference_y", "global_direct_y_hat", "global_chain_y_hat"]).list.mean().alias('ensemble'),
    pl.concat_list(forecast_cols).list.mean().alias('all_ensemble'),
    pl.concat_list(["global_direct_y_hat", "global_chain_y_hat"]).list.mean().alias('lgb_ensemble'),                   
)"""

# forecast_cols += ["ensemble", "all_ensemble", "lgb_ensemble"]

metrics_output_df = []
for col in forecast_cols:
    metrics_output_df.append(
        display_metrics(
        baseline[y].to_numpy(), 
        np.clip(baseline[col].fill_null(0).to_numpy(), a_min=0, a_max=None),
        name=col
        ).transpose()
    )

allmet = pd.concat(metrics_output_df, axis=1)
allmet.columns = allmet.iloc[0, :]
display(allmet.T.style.highlight_min(color="blue"))

: 

In [ ]:
baseline

In [ ]:
allmet

In [ ]:
rmsse_df = allmet.loc['rmse'] 
rmsse_df /= rmsse_df['reference_y']
rmsse_df = rmsse_df.to_frame('rmsse').reset_index()
rmsse_df

In [ ]:
import seaborn as sns 
import matplotlib.pyplot as plt

rmsse_df = rmsse_df.loc[~rmsse_df["fname"].isin(['reference_y', 'all_ensemble', 'ensemble', 'lgb_ensemble', "local_direct_y_hat", "NBEATSx", "NHITS"])]
rmsse_df['fname'] = np.where(rmsse_df['fname'] == "arithmetic_forecast_ensamble", "statistical_ensemble", rmsse_df['fname'])
rmsse_df['fname'] = np.where(rmsse_df['fname'] == "global_direct_y_hat", "LightGBM Direct Forecast", rmsse_df['fname'])
rmsse_df['fname'] = np.where(rmsse_df['fname'] == "global_chain_y_hat", "LightGBM Chained Forecast", rmsse_df['fname'])

plt.figure(figsize=(13, 5))
sns.barplot(rmsse_df.sort_values(by="rmsse"), x="rmsse", y="fname",
            errorbar=("pi", 50), capsize=.4,
    err_kws={"color": ".5", "linewidth": 2.5},
    linewidth=2.5, edgecolor=".5",
    palette="viridis",
)

plt.axvline(1.0, label="Baseline", color="black", linestyle='--')
plt.legend()

In [ ]:
plt.figure(figsize=(13, 5))
sns.barplot(rmsse_df.loc[rmsse_df.index.isin(list(filter(lambda x : "global" in x, rmsse_df.index)))].sort_values(by="rmsse"), x="rmsse", y="fname",
            errorbar=("pi", 50), capsize=.4,
    err_kws={"color": ".5", "linewidth": 2.5},
    linewidth=2.5, edgecolor=".5",
    palette="viridis",
)

plt.axvline(1.0, label="Baseline", color="black", linestyle='--')
plt.legend()

In [ ]:
dt = baseline['date'].unique()
split_dt = np.array_split(dt, 5)
bycut = []
for sdt in split_dt:
    metrics_output_df = []
    b_cut = baseline.filter(pl.col("date").is_in(sdt))
    for col in forecast_cols:
        metrics_output_df.append(
            display_metrics(
            baseline[y].to_numpy(), 
            np.clip(baseline[col].fill_null(0).to_numpy(), a_min=0, a_max=None),
            name=col
            ).transpose()
        )

    allmet = pd.concat(metrics_output_df, axis=1)
    allmet.columns = allmet.iloc[0, :]
    rmsse_df = allmet.loc['rmse'] 
    rmsse_df /= rmsse_df['reference_y']
    rmsse_df = rmsse_df.to_frame('rmsse')
    rmsse_df['date_range'] = str(b_cut['date'].min()) + ' - ' + str(b_cut['date'].max())
    # display(allmet)
    bycut.append(rmsse_df)

import seaborn as sns 
import matplotlib.pyplot as plt

data_ = pd.concat(bycut).reset_index()
data_ = data_.loc[~data_["fname"].isin(['reference_y', 'all_ensemble', 'ensemble', 'lgb_ensemble', "local_direct_y_hat", "NBEATSx", "NHITS"])]
data_['fname'] = np.where(data_['fname'] == "arithmetic_forecast_ensamble", "statistical_ensemble", data_['fname'])
data_['fname'] = np.where(data_['fname'] == "global_direct_y_hat", "LightGBM Direct Forecast", data_['fname'])
data_['fname'] = np.where(data_['fname'] == "global_chain_y_hat", "LightGBM Chained Forecast", data_['fname'])

plt.figure(figsize=(13, 5))
sns.barplot(data_.sort_values(by="rmsse"), x="date_range", y="rmsse", hue="fname",     errorbar=("pi", 50), capsize=.4,
    err_kws={"color": ".5", "linewidth": 2.5},
    linewidth=2.5, edgecolor=".5",
    palette="viridis",
)

plt.axhline(1.0, label="Baseline", color="black", linestyle='--')
plt.legend()

In [ ]:
import matplotlib.pyplot as plt 

uids = baseline['station'].unique()
n = 6
choice = np.random.choice(uids, (n, ))

# Création de la figure et des sous-graphiques
fig, axs = plt.subplots(3, 2, figsize=(20, 16))
axs = axs.ravel()  # Aplatir le tableau 2D des axes pour itération facile

# Tracer les graphiques
for i, uid in enumerate(uids, start=0):
    if i  < n :
        subset_tr = train_data.filter(
            pl.col(ts_uid) == uid, 
            pl.col('date') >= pl.datetime(2021, 1, 1), 
            pl.col('date') < subset['date'].min()
            ).sort(by='date')
        subset = baseline.filter(pl.col(ts_uid) == uid).sort(by='date')
        axs[i].plot(subset_tr["date"], subset_tr['y_copy'], label="train", color="brown", alpha=0.8)
        axs[i].scatter(subset["date"], subset['y'], label="real", color="blue", alpha=1, marker="*")
        for col in list(filter(lambda x : 'y_hat' in x ,forecast_cols)):
            axs[i].scatter(subset["date"], subset[col], label=col, alpha=0.5, marker="x")
        axs[i].axvline(subset['date'].min())
        axs[i].set_title(f'Graph for ID {uid}')
        axs[i].legend()

# Ajuster l'espacement entre les sous-graphiques
plt.tight_layout()
plt.show()

overall_valid =  baseline.group_by('date').agg([pl.col(col).sum() for col in forecast_cols])
overall_hist =  train_data.filter(
            pl.col('date') >= pl.datetime(2021, 1, 1), 
            pl.col('date') < subset['date'].min()
            ).sort(by='date').group_by('date').agg(pl.col(y).sum())

overall_realise =  train_data.filter(
            pl.col('date') >= subset['date'].min()
            ).sort(by='date').group_by('date').agg(pl.col(y).sum())

plt.figure(figsize=(25, 6))
plt.scatter(
    overall_hist['date'], overall_hist['y'], label="train", marker="o"
)
plt.scatter(
    overall_realise['date'], overall_realise['y'], label="realise", marker="o"
)
for col in forecast_cols:
    plt.scatter(
        overall_valid['date'], overall_valid[col], label=f"{col}_valid", marker="*"
    )
plt.legend()